In [6]:
#This Project will read the amount invested on any given day since 2015 from an excel sheet and find the total amount if the investment is
#compounded quartely with the interest rate updated in same excel but different sheet across year


import pandas as pd
import datetime
import math

#this command will suppress the warning or error message. Best to ensure that we resolve the warning or error message
#pd.options.mode.chained_assignment = None 


# move the location of the excel sheet
excel_file = 'SIP.xlsx' #if stored within the same Jupyter folder
#excel_file = '/Users/jbanerjee/Desktop/SIP2.xlsx' #if the excel sheet is stored at any other location

#read 2 sheets from the excel file - Main(contains all investments done since 2015 across days) and Interest (year - interest rate)
sip = pd.read_excel(excel_file, sheet_name = ['Main','Interest'])

#change them to list so that it can be converted to dataframe
table1 = sip['Main'].values.tolist()
table2 = sip['Interest'].values.tolist()

#change to dataframe
df = pd.DataFrame(table1[1:], columns=table1[0])
dfI = pd.DataFrame(table2[1:10], columns=table2[0])

#renaming column index on dfI dataframe to a name that will be later generated for the df dataframe
dfI = dfI.rename(columns={'Year': 'Date-Year', 'Rate': 'Interest Rate' })


#select only those 2 columns that is required to do the job from main df dataframe
df = df.iloc[:, [0,7]]

#The date field on the main sheet should have a datetime type or else the script will not run. A check is done in the next step and if this 
#field is any of int or float etc and if so it passess a default value 1-1-2000 that the user can later look into

date_time_str = '2000-01-01 00:00:00.000000'
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')


#Change Date value to 2000-01-01 if it is str or float or int or bool or complex or bytes datatype
df['Date'] = [date_time_obj if (type(x) == str or type(x) == float or type(x) == int or type(x) == bool or type(x) == complex or type(x) == bytes) else x for x in df['Date']]


#Change Invested value to 0 if it is str or bool or complex or bytes datatype from the excel sheet
df['Invested in market from Salary'] = [0 if (type(x) == str or type(x) == bool or type(x) == complex or type(x) == bytes) else x for x in df['Invested in market from Salary']]


#Change Date to Qtr
df['Date-Qtr'] = df['Date'].dt.to_period("Q")


#get all unique Qtr values. Now we need to find the sum of the invsted amount done through each of the quarters
a = df['Date-Qtr'].unique()


#initialise list b and z. We are about to define to new dataframe columns year-quarter and amount invested in that quarter
b = [] #will have the year-quarter like 2017Q3
z = [] #will have the amount of money invested in that quarter of that year


#this for loop will filter all the rows for which 
for i in range (len(a)):
    
    #increase the length of the list b and z by 1
    b = b + ['']
    z = z + ['']
    
    #concatenate the unique date with the word sum. This becomes my new variable
    b[i] = str(a[i])+"sum"
    
    #create a dynamic variable name same as Qtr of the year and store the sum in it
    name = b[i]
    

    #The loc command will filter out and group the data year and quarter wise. The whole operation is done under df['Date-Qtr'].unique() so 
    #the number of columns now reduces (drastically) to contain only unique year-qtr values like 2017Q2
    
    #.sum() will sum all values of that Qtr and store it in a variable
    value = df.loc[df['Date-Qtr'] == a[i], 'Invested in market from Salary'].sum()
    
    #How to create a variable from a variable
    #we need to store each of the quartely amount into a variable which can be used later to process
    #define a dictionary field with the name field as the field that is derieved from year-qtr concatenated with the word sum
    #the value field contains the quartely amount invested
    dict1 = {name: value}
    
    #we need to move the quartely amount invested into a variable so that we can create a dataframe
    #create variable z that will contain total amount invested for that quarter. We can access the value from the dictionary variable
    z[i] = dict1[b[i]]


#create a new dataframe with the new data and unique year-qtr rows
dfn = pd.DataFrame({"Date-Qtr": a, "Quarter":b, "Total Amount Invested":z})

#modify the new dataframe to have Year at the beginning and define a new column Interest1 with default value 0.
#Interest1 will later be modified to capture the values of rate of interest from the excel sheet. See dfI dataframe for more
dfn = pd.DataFrame({"Date-Year": dfn['Date-Qtr'].dt.year, "Date-Qtr": a, "Quarter":b, "Total Amount Invested":z,})


#Refer to stack overflow for more - https://stackoverflow.com/questions/65130977/how-to-create-a-new-column-on-an-existing-dataframe-based-on-values-of-another-d/65131058#65131058
#dfn['Interest'] = dfn['Date-Year'].replace(dfI.set_index('Date-Year').squeeze()) 
dfn['Interest'] = dfn['Date-Year'].replace(dict(dfI.to_numpy()))
#dfn = dfn.merge(dfI).rename(columns={'Interest Rate':'Interest'})


#Calculate number of quarters until now since investment and create a new column Number-of-Qtrs in the dataframe dfn
def function1(i):
    #col = str(i)
    
    date_time_str1 = '2021-01-01 14:50:30.190267'
    #now = datetime.datetime.strptime(date_time_str1, '%Y-%m-%d %H:%M:%S.%f')
        
    now = datetime.datetime.now()
    qtrvar = math.ceil(now.month/3.)
    yearvar = math.ceil(now.year)
    qtrdiff = (yearvar - a.year)*4 + (qtrvar - a.quarter)
    test = 5
    return qtrdiff


#Creating new column Number-of-Qtrs through the function1
dfn['Number-of-Qtrs'] = dfn['Date-Qtr'].apply(function1)

#Creating new column Amount = (1+ ((int rate)/4)/100) to the power Number-of-Qtrs * Principal amount
dfn['Amount'] = ((1+dfn['Interest']/400)**dfn['Number-of-Qtrs'])*dfn['Total Amount Invested']

#Changing amount to int for readability
dfn['Amount'] = dfn['Amount'].astype(int) 

#Summing up all the Amounts to get the overall number
currentbalance = dfn['Amount'].sum()

#Sort the year-quarter values in descending order for readability purpose
dfn = dfn.sort_values(by=['Quarter'])


## Export dataframe to excel without deleting current sheets of excel
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = 'SIP.xlsx'

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl') #version 3.0.4
writer.book = book


df3 = pd.DataFrame(dfn)
df3.to_excel(writer, sheet_name = 'Python')

srow = (len(dfn)) + 2
scol = len(dfn.columns)
df4 = pd.DataFrame({'Data': [currentbalance]})
df4.to_excel(writer, sheet_name='Python', header=None, index=False,startcol=scol,startrow=srow)


print (currentbalance,"\n",dfn)

#print (currentbalance,"\n",dfn[["Date-Qtr","Number-of-Qtrs"]])

#The following 2 commands are commented, uncomment it when required to create the sheets on excel 
#writer.save()
#writer.close()


7404288 
     Date-Year Date-Qtr    Quarter  Total Amount Invested  Interest  \
0        2015   2015Q1  2015Q1sum                   0.00      8.20   
1        2015   2015Q2  2015Q2sum                   0.00      8.20   
2        2015   2015Q3  2015Q3sum                   0.00      8.20   
3        2015   2015Q4  2015Q4sum                   0.00      8.20   
4        2016   2016Q1  2016Q1sum               26000.00      8.20   
5        2016   2016Q2  2016Q2sum              168000.00      8.20   
6        2016   2016Q3  2016Q3sum              535000.00      8.20   
7        2016   2016Q4  2016Q4sum              742000.00      8.20   
8        2017   2017Q1  2017Q1sum             1060000.00      7.75   
9        2017   2017Q2  2017Q2sum              735000.00      7.75   
10       2017   2017Q3  2017Q3sum              457400.00      7.75   
11       2017   2017Q4  2017Q4sum              667000.00      7.75   
12       2018   2018Q1  2018Q1sum              358183.02      7.50   
13       2